# Lab 5. Vector and Matrix Backpropogation
by Domrachev Ivan, B20-Ro-01

In [1]:
from backprop import SoftMax, ReLU, Linear
import numpy as np

## Part 1. Vectorized softmax function

> Note: accidentally, I've implemented this functionality in the previous lab assignment. Here is the copy of corresponding charpter from there

All the logic is implemented in the file [backprop.py](./backprop.py). There is an abstract class Node, which implements all the expected logic except for the function itself and the jacobian matrix calculation.

In [2]:
sm = SoftMax(5)

# Random input values (x itself, and assumed partial derivative)
x_input = np.array([0, 1, 2, 3, 4])
dL_dy = np.array([1, 2, 3, 2, 1])

# Forward call
y_value = sm.forward(x_input)

# Backpropogation
dL_dx = sm.backward(dL_dy)

y_value, dL_dx

(array([0.01165623, 0.03168492, 0.08612854, 0.23412166, 0.63640865]),
 array([-0.00510617,  0.01780491,  0.1345273 ,  0.13156147, -0.27878751]))

The verification was described in the previous lab, the code (as well as outputs) haven't changed, so let's omit that here.

## Part 2. ReLU

The function ReLU, as well as the partial derivative, were implemented according to the lecture. Note that despite derivative $\frac{\partial ReLU}{\partial x} (0)$ was chosen, it was assigned to $0$ in my implementation. 

In [3]:
relu = ReLU(5)

# Random input values (x itself, and assumed partial derivative)
x_input = np.array([0, -1, 2, -1, 4])
dL_dy = np.array([1, 2, 3, 2, 1])

# Forward call
y_value = relu.forward(x_input)

# Backpropogation
dL_dx = relu.backward(dL_dy)

y_value, dL_dx

(array([0, 0, 2, 0, 4]), array([0., 0., 3., 0., 1.]))

It is easy to see from that example that `ReLU` function works correctly.

## Part 3. Linear layer

To implement linear node $Y = WX$ in my structure, I decided to make the following assumptions:
1. The weigts matrix `W` is an inner parameter of the class, which could be accessed via a getter. It's partial derivative is also stored inside it.
2. Despite there are functions to compute derivatives w.r.t. `W` and `X`, I decided to override the functoin `forward` with dot products from `numpy`, because this approach is more efficient

You can explore the implementation in `backprop.py` file.
> *Note. To compute the jacobian $\frac{\partial Y}{\partial X}$ and $\frac{\partial Y}{\partial W}$, the key note is that:*
> 1. *$\frac{\partial Y}{\partial X}_{i, j}$ is a zero matrix, except for the $j$-th column, which equals to:*
> $$\frac{\partial Y}{\partial X}_{i, j, l, j} = W_{l, j}$$
> 2. *Similarly, $\frac{\partial Y}{\partial W}_{i, j}$ is a zero matrix, except for the $i$-th raw, which equals to:*
> $$\frac{\partial Y}{\partial W}_{i, j, j, l} = X_{j, l}$$

In [4]:
input_dim=(2, 3)
output_dim=(5, 3)
relu = Linear(input_dim, output_dim)

# Random input values (x itself, and assumed partial derivative)
x_input = np.random.rand(*input_dim)
dL_dy = np.random.rand(*output_dim)

# Forward call
y_value = relu.forward(x_input)

# Backpropogation
dL_dx = relu.backward(dL_dy)
dL_dw = relu._W_pd

print("Forward output:")
print(y_value)
print("Partial derivatives over X:")
print(dL_dx)
print("Partial derivatives over W:")
print(dL_dw)

Forward output:
[[0.31835014 0.70391377 0.56740485]
 [0.30103851 0.62169701 0.49888295]
 [0.25428618 0.52141161 0.41820394]
 [0.33700738 0.52600585 0.41277896]
 [0.3368338  0.68186608 0.54641195]]
Partial derivatives over X:
[[1.01291771 0.84933338 1.74488986]
 [0.9372118  0.7419824  1.69668606]]
Partial derivatives over W:
[[0.30063048 0.97925034]
 [0.35701621 0.69264083]
 [0.20792399 1.0302545 ]
 [0.52375687 1.31859524]
 [0.34171273 0.60522588]]


Also note that calculations of back propogation wia jacobians lead to the same result:

In [5]:
dL_dx_jac = relu.backward(dL_dy, use_jacobian=True)
dL_dw_jac = relu._W_pd

print("Jacobian dL/dx:")
print(dL_dx_jac)

print("Jacobian dL/dw")
print(dL_dw_jac)

Jacobian dL/dx:
[[1.01291771 0.84933338 1.74488986]
 [0.9372118  0.7419824  1.69668606]]
Jacobian dL/dw
[[0.30063048 0.97925034]
 [0.35701621 0.69264083]
 [0.20792399 1.0302545 ]
 [0.52375687 1.31859524]
 [0.34171273 0.60522588]]
